This is a code written in google colab

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [6]:
# !pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1v64k51u
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1v64k51u
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 4664a4ef472c35ed55ab1a53c458aa48e6f9919d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# %load_ext nvcc_plugin

In [ ]:
# %reload_ext nvcc_plugin

In [4]:
%%writefile kernel.cu
#include <stdio.h>
#include <stdlib.h>
#include <curand.h>
#include <curand_kernel.h>
#include <chrono>
#include <random>
#include <iostream>
#include <bits/stdc++.h>
#include <math.h>
#include <fstream>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

/*** GPU functions ***/

// Update acceleration of particles
__device__ void get_acc_kernel(double *p, double *m, double *a, double G, int N) {
  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  // Accleration (x, y, z) for plaent with id tid
  double x = 0;
  double y = 0;
  double z = 0;

  for(int i=0; i<N; i++){
    if(i != tid){
      // Get difference in position of neighboring particle
      double dx = p[0 + i * 3] - p[0 + tid * 3];
      double dy = p[1 + i * 3] - p[1 + tid * 3];
      double dz = p[2 + i * 3] - p[2 + tid * 3];

      // Calculate inverse with softening length (0.1) -- Part to account for particles close to eachother
      double inv = pow(pow(dx, 2) + pow(dy, 2) + pow(dz, 2) + pow(0.1, 2), -1.5);

      x = x + (m[i] * dx / inv);
      y = y + (m[i] * dy / inv);
      z = z + (m[i] * dz / inv);
    }
  }

  // Adjust with Newton's Gravitational constant
  x = x * G;
  y = y * G;
  z = z * G;

  // Assign new x,y,z accelerations to "a"
  a[0 + tid * 3] = x;
  a[1 + tid * 3] = y;
  a[2 + tid * 3] = z;
}

// Update velocity of singular particle (used each half kick)
__device__ void get_vel_kernel(double *v, double *a, double td) {
  int tid = threadIdx.x + blockIdx.x * blockDim.x;
  v[0 + tid * 3] = v[0 + tid * 3] + (a[0 + tid * 3] * td / 2.0);
  v[1 + tid * 3] = v[1 + tid * 3] + (a[1 + tid * 3] * td / 2.0);
  v[2 + tid * 3] = v[2 + tid * 3] + (a[2 + tid * 3] * td / 2.0);
}

  // Update position of singular particle (drift)
__device__ void get_pos_kernel(double *p, double *v, double *data, double td, int N, int i) {
  int tid = threadIdx.x + blockIdx.x * blockDim.x;
  p[0 + tid * 3] = p[0 + tid * 3] + (v[0 + tid * 3] * td);
  p[1 + tid * 3] = p[1 + tid * 3] + (v[1 + tid * 3] * td);
  p[2 + tid * 3] = p[2 + tid * 3] + (v[2 + tid * 3] * td);

  data[N + (i * N * 3 + 3 * tid + 0)] = p[0 + tid * 3];
  data[N + (i * N * 3 + 3 * tid + 1)] = p[1 + tid * 3];
  data[N + (i * N * 3 + 3 * tid + 2)] = p[2 + tid * 3];
}

// Run N-body simulation
__global__ void generate_data_kernel(double *p, double * v, double *m, double *a, double *data, int timesteps, double td, int G, int N) {
  // Get acceleration of particles
  get_acc_kernel(p, m, a, G, N);
  __syncthreads();

  // Loop for number of timesteps --> timestep 0 already complete
  for(int i = 1; i < timesteps; i++){
    // Use leapfrog integration
    // 1) First half kick --> update velocities
    get_vel_kernel(v, a, td);
    __syncthreads();

    // 2) Drift --> update positions
    get_pos_kernel(p, v, data, td, N, i);
    __syncthreads();

    // 3) update acceleration with new positions
    get_acc_kernel(p, m, a, G, N);
    __syncthreads();

    // 4) Second half od kick --> update velocities again
    get_vel_kernel(v, a, td);
    __syncthreads();
  }
}

/*** CPU functions ***/

// Returns data (mass and all positions) from N-body simulation
double* n_body(int N, double G, double td, int timesteps) {
  // Array of random starting positions of particles (N x (x,y,z))
  double* particle_pos = new double[N*3];
  double* d_particle_pos;
  // Array of random velocities of particles
  double* particle_vel = new double[N*3];
  double* d_particle_vel;
  // Array of random masses of particles
  double* particle_mass = new double[N];
  double* d_particle_mass;
  // Array of random masses of particles
  double* particle_acc;
  double* d_particle_acc;
  // Array of positions of particles over all timesteps
  double* data = new double[N * 3 * timesteps + N];
  double* d_data;

  // Allocate memory
  particle_acc = (double*)malloc((N*3)* sizeof(double));
  particle_pos = (double*)malloc((N*3)* sizeof(double));
  particle_vel = (double*)malloc((N*3)* sizeof(double));
  particle_mass = (double*)malloc(N * sizeof(double));
  data = (double*)malloc((N * 3 * timesteps + N) * sizeof(double));

  cudaMalloc(&d_particle_mass, N * sizeof(double));
  cudaMalloc(&d_particle_pos, N * 3 * sizeof(double));
  cudaMalloc(&d_particle_vel, N * 3 * sizeof(double));
  cudaMalloc(&d_particle_acc, N * 3 * sizeof(double));
  cudaMalloc(&d_data, (N * 3 * timesteps + N) * sizeof(double));

  srand(811);

  // Fill array of masses of particles
  for(int i=0; i<N; i++){
    particle_mass[i] = rand()/double(RAND_MAX)*1.f+0.f;
    data[i] = particle_mass[i];
  }

  // Fill array of random starting velocities & positions for each particle
  for(int i= 0; i<N; i++){
    if (i % 4 == 1){
      particle_pos[0 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+0.f;
      particle_pos[1 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+0.f;
      particle_pos[2 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+2.f;

      particle_vel[0 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+0.f;
      particle_vel[1 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+0.f;
      particle_vel[2 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+1.f;
    }
    else if (i % 4 == 2){
      particle_pos[0 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+0.f;
      particle_pos[1 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+2.f;
      particle_pos[2 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+2.f;

      particle_vel[0 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+0.f;
      particle_vel[1 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+1.f;
      particle_vel[2 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+0.f;
    }
    else if (i % 4 == 3){
      particle_pos[0 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+2.f;
      particle_pos[1 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+2.f;
      particle_pos[2 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+2.f;

      particle_vel[0 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+1.f;
      particle_vel[1 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+1.f;
      particle_vel[2 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+1.f;
    }
    else{
      particle_pos[0 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+2.f;
      particle_pos[1 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+0.f;
      particle_pos[2 + 3 * i] = (double)rand()/(double)RAND_MAX*-2.f+2.f;

      particle_vel[0 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+1.f;
      particle_vel[1 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+0.f;
      particle_vel[2 + 3 * i] = (double)rand()/(double)RAND_MAX*-1.f+1.f;
    }

    // Save initial particle positions
    data[(0 + 3 * i) + N] = particle_pos[0 + 3*i];
    data[(1 + 3 * i) + N] = particle_pos[1 + 3*i];
    data[(2 + 3 * i) + N] = particle_pos[2 + 3*i];
  }

  // Copy variables from host to device
  cudaMemcpy(d_particle_mass, particle_mass, N * sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_particle_pos, particle_pos, N * 3 * sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_particle_vel, particle_vel, N * 3 * sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_data, data, N * 3 * timesteps * sizeof(double), cudaMemcpyHostToDevice);

  // Determine number of blocks based on N
  int n_blocks;
  if  (N % 1024 == 0){
    n_blocks = N / 1024;
  }
  else{
    n_blocks = 1 + floor(N / 1024);
  }

  // Deterine number of threads based on N
  int n_threads;
  if (N <= 1024){
    n_threads = N;
  }
  else{
    n_threads = 1024;
  }

  // Call GPU kernel to run simulation
  generate_data_kernel <<< n_blocks, n_threads >>> (d_particle_pos, d_particle_vel, d_particle_mass, d_particle_acc, d_data,timesteps, td, G, N);

  // Copy varibles device to host --> maybe just need
  cudaMemcpy(data, d_data, (N * 3 * timesteps +N) * sizeof(double), cudaMemcpyDeviceToHost);

  // Free memory
  cudaFree(d_particle_pos);
  cudaFree(d_particle_vel);
  cudaFree(d_particle_acc);
  cudaFree(d_particle_mass);
  cudaFree(d_data);
  free(particle_pos);
  free(particle_vel);
  free(particle_acc);
  free(particle_mass);

  // Return particles masses and all positions --> data
  return data;
}


int main(int argc, char** argv) {
  // Number of particles
    //int N = 100;
  int N = atoi(argv[1]);
  // Newton's Gravitational Constant
  double G = pow(6.67 * 10, -11);

  // Start time of simulation
  auto t_start = std::chrono::high_resolution_clock::now();

  //  Set number of timesteps (number of interations for simulation)
  double td = 0.01;
  int timesteps = atoi(argv[2]);
  //int timesteps = 50;

  // Run N-body simulation
  double* data = n_body(  N, G, td, timesteps);

  // End time of simulation
  auto t_end = std::chrono::high_resolution_clock::now();
  // Runtime duration in seconds
  auto total_time = std::chrono::duration_cast<std::chrono::microseconds>(t_end - t_start) * 0.000001;

  // Write to output file
  std::ofstream output_file;
  output_file.open("output_cu.txt");
  output_file << "Positions of " << N << " particles over " << timesteps <<" timesteps: \n";

  // Write masses
  for(int i=0; i < N; i++){
    if (i == N-1) {
      output_file << data[i] << "\n";
    } else {
      output_file << data[i] << ", ";
    }
  }

  // Write runtime duration
  output_file << total_time.count() << "\n";

  // Write positions
  int curr_step = 0;
  for(int i=N; i < timesteps * N * 3 + N; i++){
    if (curr_step == (N*3)-1) {
      output_file << data[i] << "\n";
      curr_step = 0;
    } else {
      curr_step++;
      output_file << data[i] << ", ";
    }
  }
  output_file.close();

  // Free data
  free(data);

  return 0;
}


Writing kernel.cu


In [5]:
! nvcc kernel.cu

In [6]:
! nvcc kernel.cu -o test1

In [ ]:
! ./test1 100 40

In [ ]:
%%writefile nbody.py
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import sys

'''
* This program generates the movement of particles using the N-body problem.
* Produces an output file with particles' positions over a given number of timesteps.
'''

# Return matrix of accelertaions (x, y, z) for each particle
# p: Maxtix of particle postions (x, y, z)
# m: Array of particle masses
# G: Newton's Gravitational Constant
# N: Number of particles (bodies)
def getAcc(p, m, G, N):
    # new_acc is N x 3 matrix of updated accelerations (x, y, z for each particle)
    new_acc = np.zeros((len(p), 3))

    # get accleration for each particle
    for i in range(N):
        x = 0
        y = 0
        z = 0
        # Calculate orce exerted on each particel THEN sum
        for j in range(N):
            if j != i:
                # Get difference in position of neighboring particle
                dx = p[j][0] - p[i][0]
                dy = p[j][1] - p[i][1]
                dz = p[j][2] - p[i][2]

                # Calculate inverse with softening length (0.1) -- Part to account for particles close to eachother
                inv = (dx**2 + dy**2 + dz**2 + 0.1**2)**(-1.5)

                # Update acceleration (x, y, z)
                x += m[j] * dx / inv
                y += m[j] * dy / inv
                z += m[j] * dy / inv

        # Ajust with Newton's gravitational constant
        x *= G
        y *= G
        z *= G

        # Update with new acceleration
        new_acc[i][0] = x
        new_acc[i][1] = y
        new_acc[i][2] = z

    return new_acc

# Return array with postions of particles for each timestep
# data: array of maxtix of postions of particles for each timestep
# p: particle postions (x, y, z)
def format(data, p):
    all_pos = []
    for i in range(len(p)):
        for j in range(3):
            all_pos.append(p[i][j])
    data.append(all_pos)

    return data

# Write data to output file
# data: Maxtix of postions of particles for each timestep of the simulation
# output_file: name of output file
def generateOutput(data, output_file, runtime):
    f = open(output_file, 'a')
    f.write(str(runtime) + "\n")
    for i in range(len(data)):
        f.write(str(data[i]) + "\n")
    f.close()


def main():
    ''' Generate N-body Simulation Data '''

    # Number of particles
    N = int(sys.argv[1])
    # Newton's Gravitational Constant
    G = 6.67 * 10**-11
    # Random number generator seed
    np.random.seed(811)
    # Path for output file
    output = "output_py.txt"

    # Start timer -> for performance comparision
    t_start = time.time()

    # Create N x 3 matrix of random starting postion of particles (size N) -> each partile has x,y,z corrdinate
    particle_pos = np.random.uniform(-2.5, 2.5, N *3)
    particle_pos = np.reshape(particle_pos, (-1,3))

    # Data that will be outputed
    data = format([], particle_pos)

    # Create N x 3 matrix of random starting velocities for each particle
    particle_vel = np.random.randn(N, 3)

    # Create N x 1 array of masses of particles
    particle_mass = np.random.rand(N,1)

    # Get starting accelerations of particles (N x 3 matrix)
    particle_acc = getAcc(particle_pos, particle_mass, G, N)

    # Set number of timesteps (number of interations for simulation)
    td = 0.01 # Timestep duration
    timesteps = int(sys.argv[2]) # Number of timesteps

    f = open(output, 'w+')
    f.write("Positions of " + str(N) + " particles over " + str(timesteps) + " timesteps: \n")
    f.write(str(particle_mass.tolist()) + "\n")
    f.close()

    # Loop for number of timesetps
    for i in range(timesteps): # change 5 to timesteps
        # Leapfrog integration
        # 1) first half kick
        particle_vel += particle_acc * (td / 2.0)

        # 2) Drift --> Update positions of all particles
        particle_pos += particle_vel * td

        # 3) Get new accleration for each particle
        particle_acc = getAcc(particle_pos, particle_mass, G, N)

        # 4) Second half of kick --> update velocities
        particle_vel += particle_acc * (td / 2.0)

        # 6) Append to data that will be outputed
        data = format(data, particle_pos)

    # Get end time of simuation
    t_end = time.time()

    # Write to output file
    generateOutput(data, output, t_end - t_start)

main()

Writing nbody.py


In [ ]:
! python nbody.py 100 50

In [ ]:
%%writefile simulator.py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sys

'''
* This program visulizes the N-body simulation from a dataset
'''

def main():
    # Path for output file
    f = open(sys.argv[1], 'r')
    content = f.readlines()

    # Recreate N x 1 array of particle masses
    masses = content[1].replace("[", "").replace("]", "").replace(",", "").split()
    particle_mass = np.array(masses).astype(float)

    # Visulization setup
    grid = plt.GridSpec(1, 1, wspace=0.0, hspace=0.0)
    ax1 = plt.subplot(grid[0:2,0])
    ax1.set_facecolor("blue")

    # Parse file to create N x 3 matrix of current particle positions THEN plot
    for line in content[3:]:
        # Get particle positions
        particle_pos = np.array(line.replace("[", "").replace("]", "").replace(",", "").split()).astype(float)
        N = int(len(particle_pos) / 3) # each particle has (x, y, z)
        particle_pos = particle_pos.reshape(N, 3)

        # Plot
        plt.sca(ax1)
        plt.cla()
        plt.scatter(particle_pos[:, 0], particle_pos[:, 1], s=100*particle_mass, color = 'lime', edgecolor='purple')
        ax1.set(xlim=(-3, 3), ylim=(-3, 3))
        ax1.set_aspect('equal', 'box')
        plt.title(label= "N-Body Simulation", fontsize=20, color='black')
        plt.pause(0.001)

    f.close()

main()

Writing simulator.py


In [ ]:
! python simulator.py output_py.txt

Figure(640x480)
Traceback (most recent call last):
  File "simulator.py", line 42, in <module>
    main()
  File "simulator.py", line 32, in main
    plt.sca(ax1)
  File "/usr/local/lib/python3.8/dist-packages/matplotlib/pyplot.py", line 858, in sca
    raise ValueError("Axes instance argument was not found in a figure")
ValueError: Axes instance argument was not found in a figure


In [ ]:
! python simulator.py output_cu.txt

Figure(640x480)
Traceback (most recent call last):
  File "simulator.py", line 42, in <module>
    main()
  File "simulator.py", line 32, in main
    plt.sca(ax1)
  File "/usr/local/lib/python3.8/dist-packages/matplotlib/pyplot.py", line 858, in sca
    raise ValueError("Axes instance argument was not found in a figure")
ValueError: Axes instance argument was not found in a figure


In [ ]:
%%writefile evaluate.py
import sys
import os
import matplotlib.pyplot as plt

'''
* This program generates a graph comparing runtime of the serial and parallel
implemetations of the N-body simulation.
* N increases in powers of 2
* Number of timsteps is constant (150).
'''

def runSim(N, iterations):
    py_runtimes = []
    cu_runtimes = []
    n_vals = []
    for i in range(iterations):
        n_vals.append(N)
        print("Running evaluation with N = " + str(N))

        # Get serial nbody runtime
        os.system("python3 nbody.py " + str(N) + " 150")
        f_py = open("../data/output_py.txt", 'r')
        content_py = f_py.readlines()
        py_runtimes.append(float(content_py[2]))
        f_py.close()

        # Get parallel nbody runtime
        os.chdir("../build")
        os.system("./nbody " + str(N) + " 150")
        os.chdir("../python_code")
        f_cu = open("../data/output_cu.txt", 'r')
        content_cu = f_cu.readlines()
        cu_runtimes.append(float(content_cu[2]))
        f_cu.close()

        # Increment number of N (powers of 2)
        N *= 2

    return py_runtimes, cu_runtimes, n_vals

def graph(py, cu, n_vals):
    plt.xscale("log")
    plt.title(label="Runtime Comparision", fontsize=20, color='black')
    plt.plot(n_vals, py, label="Serial Implementation")
    plt.plot(n_vals, cu, label="Parallel Implementation")
    plt.xlabel('Number of Particles (log scale)')
    plt.ylabel('Runtime (sec)')
    plt.legend()
    plt.savefig("../figures/Evaluate_" + str(len(n_vals)) + "_iter.png")

def main():
    ''' Run Evaluation of Serial vs Parallel Implementaitons '''

    iterations = int(sys.argv[1])
    py, cu, n_vals= runSim(2, iterations)
    graph(py, cu, n_vals)

main()

Writing evaluate.py
